In [1]:
from skyfield.api import load, wgs84, Angle
import pickle
from skyfield.api import  Topos
from skyfield.positionlib import ICRF
from skyfield.data import mpc
from skyfield.constants import GM_SUN_Pitjeva_2005_km3_s2 as GM_SUN
import pandas as pd

# Planets

In [2]:
planets=[
    'mercury barycenter',
    'venus barycenter',
    'mars barycenter',
    'jupiter barycenter',
    'saturn barycenter',
    'uranus barycenter',
    'neptune barycenter',
    'pluto barycenter'
]
ts = load.timescale()
time_p = ts.utc(2018, 1, range(1,365*3+2),12,20)

In [3]:
time_p[365*3].utc_jpl()

'A.D. 2020-Dec-31 12:20:00.0000 UTC'

In [5]:
eph = load('de421.bsp')

In [6]:
earth = eph['earth']
#moon = eph['moon']
gangtok = earth + wgs84.latlon( 27.3314 , 88.6138)
gg = gangtok.at(time_p)
P=[]
for i in planets:
    P.append(eph[i])

In [7]:
astro = gg.observe(P[0]).apparent()
alt, az, dist = astro.altaz()

In [8]:
sun = eph['sun']

In [9]:
alt.degrees[0]

-32.840996851510724

In [10]:
planet_c_dist = {}

for planet, p_name in zip(P, planets):
    print(p_name," done ")
    apparent = gg.observe(planet).apparent()
    alt, az, dist = apparent.altaz()
    eph_planet = {'alt' : alt,
                  'az' : az,
                 'distance' : dist}
    planet_c_dist[p_name[:-11]]= eph_planet

mercury barycenter  done 
venus barycenter  done 
mars barycenter  done 
jupiter barycenter  done 
saturn barycenter  done 
uranus barycenter  done 
neptune barycenter  done 
pluto barycenter  done 


In [11]:
planet_c_dist['pluto']

{'alt': <Angle 1096 values from -05deg 07' 52.2" to -00deg 01' 47.0">,
 'az': <Angle 1096 values from 248deg 13' 39.5" to 244deg 33' 19.4">,
 'distance': <Distance [34.44992545 34.45276288 34.45531009 ... 35.13462316 35.13987256
  35.14483876] au>}

In [12]:
planet_names=[
    'mercury',
    'venus',
    'mars',
    'jupiter',
    'saturn',
    'uranus',
    'neptune',
    'pluto'
]

In [13]:
Altitude={}
Azimuth={}
Distance={}
for name in planet_names:
    Altitude[name]=planet_c_dist[name]['alt'].degrees
    Azimuth[name]=planet_c_dist[name]['az'].degrees
    Distance[name]=planet_c_dist[name]['distance'].au

In [15]:
time=time_p.utc_jpl()

In [16]:
Altitude['time']=time
Azimuth['time']=time
Distance['time']=time

In [17]:
time[365*3]

'A.D. 2020-Dec-31 12:20:00.0000 UTC'

# ceres

In [23]:
with load.open('MPCORB.excerpt.DAT') as f:
    minor_planets = mpc.load_mpcorb_dataframe(f)

print(minor_planets.shape[0], 'minor planets loaded')

4 minor planets loaded


In [24]:
row_ceres=minor_planets.iloc[0]

In [27]:
ceres = sun + mpc.mpcorb_orbit(row_ceres, ts, GM_SUN)
alt, az, dis =gg.observe(ceres).apparent().altaz()
print(alt)
print(az)
print(dis.au)

1096 values from -16deg 30' 51.9" to 39deg 31' 27.9"
1096 values from 47deg 39' 47.4" to 214deg 32' 33.1"
[1.7195909  1.71187528 1.70437557 ... 3.26702094 3.27915719 3.29121405]


In [28]:
Altitude['ceres']=alt.degrees
Azimuth['ceres']=az.degrees
Distance['ceres']=dis.au

In [30]:
Altitude['ceres']

array([-16.51440593, -15.71687632, -14.90824871, ...,  39.95992946,
        39.74593744,  39.52442374])

In [31]:
col_names=[
    'time',
    'mercury',
    'venus',
    'mars',
    'jupiter',
    'saturn',
    'uranus',
    'neptune',
    'pluto',
    'ceres'
]

In [32]:
alts = pd.DataFrame(Altitude, columns=col_names)
azs = pd.DataFrame(Azimuth, columns=col_names)
dists = pd.DataFrame(Distance, columns=col_names)

In [34]:
azs

,time,mercury,venus,mars,jupiter,saturn,uranus,neptune,pluto,ceres
0,A.D. 2018-Jan-01 12:20:00.0000 UTC,261.562742,250.618086,289.446186,285.980489,254.825931,155.424334,224.918122,248.227641,47.663171
1,A.D. 2018-Jan-02 12:20:00.0000 UTC,261.277342,250.522455,289.472231,286.631496,255.142453,158.118830,225.937191,248.636334,48.241056
2,A.D. 2018-Jan-03 12:20:00.0000 UTC,260.972223,250.438052,289.499528,287.308622,255.457894,160.895768,226.934011,249.041355,48.810133
3,A.D. 2018-Jan-04 12:20:00.0000 UTC,260.652686,250.364848,289.528171,288.013897,255.772347,163.748338,227.909125,249.442834,49.370473
4,A.D. 2018-Jan-05 12:20:00.0000 UTC,260.323403,250.302806,289.558264,288.749551,256.085907,166.668023,228.863090,249.840905,49.922152
...,...,...,...,...,...,...,...,...,...,...
1091,A.D. 2020-Dec-27 12:20:00.0000 UTC,246.951889,260.512111,140.776928,240.582919,240.920259,116.331239,213.838694,242.754339,211.030473
1092,A.D. 2020-Dec-28 12:20:00.0000 UTC,246.720430,260.189490,141.891759,241.027857,241.401765,117.606960,215.181211,243.212372,211.919883
1093,A.D. 2020-Dec-29 12:20:00.0000 UTC,246.507909,259.878862,143.038575,241.468925,241.877893,118.947102,216.493687,243.665125,212.801748
1094,A.D. 2020-Dec-30 12:20:00.0000 UTC,246.314503,259.580332,144.219039,241.906231,242.348794,120.356914,217.776374,244.112744,213.675987


In [35]:
alts.to_csv('Altitudes.csv', index= False)
azs.to_csv('Azimuths.csv', index= False)
dists.to_csv('Distances.csv', index= False)

In [86]:
from skyfield.jpllib import SpiceKernel
#kernel = SpiceKernel('wld23593.15')

In [87]:
SpiceKernel

skyfield.jpllib.SpiceKernel